<a href="https://colab.research.google.com/github/drsimonturega/ir_spec_man/blob/main/asptocsv_multi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Infrared spectral manipulation: *.asp to *.csv
### One spectra at a time
### You need to be logged into your google acount to use this code
### There are three sections of code that need to be run in sequence;
- ### Lab book code for this experiemnt
- ### Upload your .asp spectra file (.asp from a Agilent ATR-FTIR spectrometer)
- ### Extract data from *.asp to transposed *.csv

### This will provide a *.zip file for down load your orginal file and your spectra as a transposed *.csv.

In [1]:
# @title ## Lab book code for this experiemnt
# @markdown ### Enter the lab book code for this experiemnt
lab_book_code = '03072024_8' # @param {type:"string"}
import os
import shutil

os.chdir("/content")
try:
    os.mkdir(lab_book_code)
except FileExistsError:
    print(f'A folder for {lab_book_code} already exists you may over write your\
        data if you continue')
    #the lines below remove the folder! not currently implimented
    #os.chdir("..")
    #shutil.rmtree(lab_book_code)
try:
    !pwd
    os.chdir(lab_book_code)
    !pwd
except FileNotFoundError:
    print(f'try again')



/content
/content/03072024_8


## Upload your .asp spectra file (.asp from a Agilent ATR-FTIR spectrometer)



In [7]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
spec_list = list(uploaded.keys())
print(f'{spec_list}')
#spec_trans(str)
for i in range(len(spec_list)):
    spec_list[i] = str(spec_list[i])
    spec_trans = spec_list[i]
    #print(f'spec_trans')
    spec_trans = spec_trans[:-4]
    spec_list[i] = spec_trans

#spec_list = spec_list[:-4]
print(f'{spec_list}')

Saving BAS_25_001_2023-06-14T09-55-56.asp to BAS_25_001_2023-06-14T09-55-56.asp
Saving BAS_100_001_2023-06-14T09-58-46.asp to BAS_100_001_2023-06-14T09-58-46.asp
Saving BAS_Buffer_001_2023-06-13T15-54-39.asp to BAS_Buffer_001_2023-06-13T15-54-39.asp
Saving BAS_Buffer_001_2023-06-13T15-56-35.asp to BAS_Buffer_001_2023-06-13T15-56-35.asp
Saving BAS_Water_2023-06-13T15-51-57.asp to BAS_Water_2023-06-13T15-51-57.asp
User uploaded file "BAS_25_001_2023-06-14T09-55-56.asp" with length 17340 bytes
User uploaded file "BAS_100_001_2023-06-14T09-58-46.asp" with length 17358 bytes
User uploaded file "BAS_Buffer_001_2023-06-13T15-54-39.asp" with length 17357 bytes
User uploaded file "BAS_Buffer_001_2023-06-13T15-56-35.asp" with length 17340 bytes
User uploaded file "BAS_Water_2023-06-13T15-51-57.asp" with length 17342 bytes
['BAS_25_001_2023-06-14T09-55-56.asp', 'BAS_100_001_2023-06-14T09-58-46.asp', 'BAS_Buffer_001_2023-06-13T15-54-39.asp', 'BAS_Buffer_001_2023-06-13T15-56-35.asp', 'BAS_Water_202

### Extract data from *.asp to transposed *.csv

In [84]:
import os
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter
from sklearn.linear_model import LinearRegression
import statistics
from tabulate import tabulate
#from google.colab import files
import shutil


class asptospectocsv():


    def __init__(self, asp_input):
        return

    def spec_import(self, spec_sht_name):
        #importing spectra from .asp from a waters? ATR-FTIR spectrometer
        spec_inp= pd.DataFrame()
        try:
            spec_inp = pd.read_csv(spec_sht_name + '.asp')
        except PermissionError:
            print('The file we are trying to open may be open in another \
                program, please close and try again!')
        #print(spec_inp.head())
        return spec_inp

    def asptodfd(self, spec_inp, spec_sht_name):
      from numpy import log as ln

      ir_spec_raw = spec_inp

      # we need th col name as a float?
      col_head = list(ir_spec_raw.columns)
      num_spec_points = col_head[(0)]

      # build the x axis (wavenumbers) from the file format
      high_wavnum = ir_spec_raw.iloc[0,0] #highest wavenumber
      high_wavnum = float(high_wavnum)
      low_wavnum = ir_spec_raw.iloc[1,0] #lowest wavenumber
      low_wavnum = float(low_wavnum)
      num_spec_points = float(num_spec_points)
      spec_gap = ( high_wavnum - low_wavnum ) / num_spec_points #calaculate spectral gap
      wav_num = []
      wav_nums = np.arange(low_wavnum, high_wavnum, spec_gap)
      wav_num_rev=[]

      for i in reversed(np.arange(low_wavnum, high_wavnum, spec_gap)): #build the list of wave numbers
        wav_num_rev.append(i)
      #set first col in dataframe as x axis (wavenumber)
      spec_sin = pd.DataFrame()
      spec_sin['wave number / cm^-1'] = wav_num_rev

      #add y axis (transmitance) to dataframe
      spec_trans=[] # a list to transport transmitance data
      spec_trans = ir_spec_raw.iloc[5:,0] #it gets sliced
      spec_trans = spec_trans.reset_index(drop=True) #it gets reindexed
      spec_sin['spec1'] = spec_trans #spec name
      bg = pd.DataFrame.transpose(spec_sin)

      bg.rename(columns=bg.iloc[0,0:], inplace=True) # set row 0 as col titles
      bg = bg.reset_index(drop=True)
      bg = pd.DataFrame.drop(bg, axis=0, index=0)
      #print(bg)
      spec_sin = bg
      spec_sin.insert(loc = 0, column = 'sample ID', value = spec_sht_name)
      spec_sin = spec_sin.reset_index(drop=True) #it gets reindexed
      #print(spec_sin)
      return spec_sin

    def asptodfd_multi(self,  spec_list, spec_sin):
      print(f'{len(spec_list)}')
      print(f'{(spec_list)}')
      for i in range(1,(len(spec_list))):
        print(f'{(spec_list[i])}')
        spec_n = asptospectocsv.spec_import(self, spec_list[i])
        #spec_trans=[] # a list to transport transmitance data
        spec_trans = spec_n.iloc[4:,0] #it gets sliced
        spec_trans = spec_trans.reset_index(drop=True) #it gets reindexed
        spec_trans.iloc[0] = spec_list[i]
        print(type(spec_trans))
        spec_trans = pd.DataFrame(spec_trans).transpose()
        spec_trans = spec_trans.reset_index(drop=True) #it gets reindexed
        spec_sin.loc[(i+1)] = spec_trans #spec name
      print(spec_trans)
      print(spec_sin)

      return #spec_n


    def savendl(self,spec_sin, asp_input):
      print(f'what is this {spec_sin}')
      try: #output results
            spec_sin.to_csv("ir_" + asp_input + ".csv",index=False)
      except PermissionError:
            print('The file we are trying to open may be open in another \
                program, please close and try again!')
      os.chdir("/content")
      shutil.make_archive(lab_book_code, 'zip', lab_book_code)
      files.download(f'{lab_book_code}.zip')
      return

specasp = asptospectocsv(spec_list)
spec_first = spec_list[0]
spec_csv = asptospectocsv.spec_import(specasp, spec_first)

final_csv = asptospectocsv.asptodfd(specasp, spec_csv, spec_first)
loop_csv = asptospectocsv.asptodfd_multi(specasp, spec_list, final_csv)
#asptospectocsv.savendl(specasp, final_csv, spec_list)

5
['BAS_25_001_2023-06-14T09-55-56', 'BAS_100_001_2023-06-14T09-58-46', 'BAS_Buffer_001_2023-06-13T15-54-39', 'BAS_Buffer_001_2023-06-13T15-56-35', 'BAS_Water_2023-06-13T15-51-57']
BAS_100_001_2023-06-14T09-58-46
<class 'pandas.core.series.Series'>


ValueError: cannot set a row with mismatched columns

In [24]:
os.listdir()

['.config',
 'BAS_100_001_2023-06-14T09-58-46.asp',
 'BAS_Buffer_001_2023-06-13T15-54-39.asp',
 'BAS_Water_2023-06-13T15-51-57.asp',
 'BAS_Buffer_001_2023-06-13T15-56-35.asp',
 'BAS_25_001_2023-06-14T09-55-56.asp',
 'sample_data']